In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

notebook_path = Path().absolute()
sys.path.append(str(notebook_path.parent))

In [3]:
import torch
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from neural_controllers import NeuralController

/home/ubuntu/miniconda3/envs/nc_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from utils import programming_language_dataset, pca_programming_language_dataset

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

In [5]:
import torch
torch.cuda.empty_cache()

model_type = 'llama'

if model_type=='llama':
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

    language_model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="cuda"
    )

    use_fast_tokenizer = "LlamaForCausalLM" not in language_model.config.architectures
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
    tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
    model_name='llama_3_8b_it'
    
elif model_type=='gemma':

    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
    language_model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2-9b-it",
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    model_name='gemma_2_9b_it'

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.81s/it]


In [6]:
# concept_types = ['python', 'c++']
concept_types = ['python', 'javascript']
data_dir = "../data/programming"

dataset = programming_language_dataset(concept_types, tokenizer)
# dataset = pca_programming_language_dataset(concept_types, tokenizer)

Generating train split: 100%|██████████| 2360/2360 [00:00<00:00, 59167.57 examples/s]

train 500 test 500
train 500 test 500


In [7]:
controllers = {}
for concept_type in tqdm(concept_types):
    
    other_type = [k for k in concept_types if k != concept_type][0]
    
    train_data = dataset[concept_type]['train']
    test_data = dataset[concept_type]['test']
        
    language_controller = NeuralController(
        language_model,
        tokenizer,
        rfm_iters=8,
        batch_size=2,
    )
    
    language_controller.compute_directions(train_data['inputs'], train_data['labels'])
    
    controllers[concept_type] = language_controller
    

  0%|          | 0/2 [00:00<?, ?it/s]

n_components: 5
Hidden layers KA: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 5
calibrate            : False

Tuning metric: auc
Getting activations from forward passes


100%|██████████| 200/200 [03:57<00:00,  1.19s/it]


Getting activations from forward passes


100%|██████████| 50/50 [00:43<00:00,  1.14it/s]


...

Optimal M batch size: 400
Time taken for round 2: 0.010113000869750977 seconds
Optimal M batch size: 400
Time taken for round 3: 0.010039329528808594 seconds
Optimal M batch size: 400
Time taken for round 4: 0.010027647018432617 seconds
Optimal M batch size: 400
Time taken for round 5: 0.0100250244140625 seconds
Optimal M batch size: 400
Time taken for round 6: 0.010037660598754883 seconds
Optimal M batch size: 400
Time taken for round 7: 0.010019540786743164 seconds
Optimal M batch size: 400
Debug: y_true shape: (100,), pred_proba shape: (100,)
Debug: y_true unique values: [0. 1.]
Fitting RFM with reg=0.001, bw=100, center_grads=False
Fitting RFM with ntrain: 400, d: 4096, and nval: 100
Optimal M batch size: 400
Time taken for round 0: 0.007600307464599609 seconds
Optimal M batch size: 400
Time taken for round 1: 0.009608745574951172 seconds
Optimal M batch size: 400
Time taken for round 2: 0.009995222091674805 seconds
Optimal M batch size: 400
Time taken for round 3: 0.00998520851135

Optimal M batch size: 400
Time taken for round 2: 0.010351181030273438 seconds
Optimal M batch size: 400
Time taken for round 3: 0.010027885437011719 seconds
Optimal M batch size: 400
Time taken for round 4: 0.010058164596557617 seconds
Optimal M batch size: 400
Time taken for round 5: 0.010044574737548828 seconds
Optimal M batch size: 400
Time taken for round 6: 0.01003575325012207 seconds
Optimal M batch size: 400
Time taken for round 7: 0.01002645492553711 seconds
Optimal M batch size: 400
Debug: y_true shape: (100,), pred_proba shape: (100,)
Debug: y_true unique values: [0. 1.]
Fitting RFM with reg=0.001, bw=100, center_grads=False
Fitting RFM with ntrain: 400, d: 4096, and nval: 100
Optimal M batch size: 400
Time taken for round 0: 0.007104158401489258 seconds
Optimal M batch size: 400
Time taken for round 1: 0.009615421295166016 seconds
Optimal M batch size: 400
Time taken for round 2: 0.01001596450805664 seconds
Optimal M batch size: 400
Time taken for round 3: 0.009992122650146

100%|██████████| 31/31 [00:23<00:00,  1.33it/s]

Optimal M batch size: 400
Time taken for round 3: 0.010125875473022461 seconds
Optimal M batch size: 400
Time taken for round 4: 0.010051488876342773 seconds
Optimal M batch size: 400
Time taken for round 5: 0.010024547576904297 seconds
Optimal M batch size: 400
Time taken for round 6: 0.01007699966430664 seconds
Optimal M batch size: 400
Time taken for round 7: 0.010030984878540039 seconds
Optimal M batch size: 400
Debug: y_true shape: (100,), pred_proba shape: (100,)
Debug: y_true unique values: [0. 1.]
Fitting RFM with reg=0.001, bw=100, center_grads=False
Fitting RFM with ntrain: 400, d: 4096, and nval: 100
Optimal M batch size: 400
Time taken for round 0: 0.008156299591064453 seconds
Optimal M batch size: 400
Time taken for round 1: 0.009635448455810547 seconds
Optimal M batch size: 400
Time taken for round 2: 0.010007619857788086 seconds
Optimal M batch size: 400
Time taken for round 3: 0.01000356674194336 seconds
Optimal M batch size: 400
Time taken for round 4: 0.00999164581298


100%|██████████| 2/2 [10:39<00:00, 319.67s/it]


In [8]:
for concept_type in concept_types:
    try:
        controller = controllers[concept_type]
        other_type = [k for k in concept_types if k!=concept_type][0]
        controller.save(concept=f'{concept_type}_{other_type}', model_name=model_name, path='../directions/')
    except:
        print(f'{concept_type} not found')
    

# Control

In [9]:
from datasets import load_dataset
huggingface_dataset = load_dataset("greengerong/leetcode")
python_dataset = huggingface_dataset["train"]['python']
js_dataset = huggingface_dataset["train"]['javascript']


def extract_code(c):
    items = c.split("```")
    code = items[1]
    return code

In [10]:
concept_types = ['python', 'javascript']
controllers = {}

for concept_type in concept_types:
    
    controller = NeuralController(
        language_model,
        tokenizer,
        control_method='rfm',
        n_components=1
    )
    
    other_type = [k for k in concept_types if k!=concept_type][0]
    
    try:
        controller.load(
                        concept=f'{concept_type}_{other_type}', 
                        model_name=model_name, 
                        path='../directions/')
        controllers[concept_type] = controller
    except:
        print(f'{concept_type} not found')
    

n_components: 1
Hidden layers KA: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 10
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 1
calibrate            : False

Detector found
n_components: 1
Hidden layers KA: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 10
forward_batch_size   : 2
M_

/home/ubuntu/miniconda3/envs/nc_env/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [ ]:
# # concept_type = "javascript"
# concept_type = "python"

# idx=0
# js_task = extract_code(js_dataset[idx])
# python_task = extract_code(python_dataset[idx])
# prompt = f"Re-state the following program. "
# # prompt = f"Give a single, different re-writing of this program with the same function. "
# # prompt += f"The output will be judged by an expert in all programming languages. "
# prompt += f"Do not include an explanation.\n\n```{python_task}```"
# # prompt += f"Do not include an explanation.\n\n```{js_task}```"

# # prompt = f"Re-state the following program. Do not include an explanation. {python_task}."
# # prompt = f"Give a single, different re-writing of this program with the same function. "
# # prompt += f"The output will be judged by an expert in all programming languages. "
# # # prompt += f"Do not include an explanation.\n\n```{python_task}```"



# layer_id = list(range(-1, -31, -1))
# # layer_id = list(range(-1, -41, -1))
# language_controller = controllers[concept_type]
# num_new_tokens = 150

# inputs = language_controller.format_prompt(prompt)


# # rfm
# # coeff=9 # for javascript, gemma
# coeff=0.7 # for javascript, llama

# print(inputs)
# print("===== No Control =====")
# gen1 = language_controller.generate(inputs, max_new_tokens=num_new_tokens, do_sample=False)
# print(gen1[len(inputs):])
# print()
# print(f"===== + {concept_type} Control =====")
# gen2 = language_controller.generate(inputs, layers_to_control=layer_id, control_coef=coeff, 
#                             max_new_tokens=num_new_tokens, do_sample=False)
# print(gen2[len(inputs):])
# print()

# Choose concept type: "python" or "javascript"
concept_type = "python"

dataset = python_dataset if concept_type == "python" else js_dataset
controller = controllers[concept_type]

layer_id = list(range(-1, -31, -1))  # or customize
coeff = 0.7  # adjust this as needed
num_new_tokens = 150

# Test multiple indices (e.g., first 5 examples)
for idx in range(5):
    task_code = extract_code(dataset[idx])

    # Choose the kind of prompt you want to test
    prompt = f"Re-state the following program. Do not include an explanation.\n\n```{task_code}```"
    # prompt = f"Give a single, different re-writing of this program with the same function. Do not include an explanation.\n\n```{task_code}```"

    inputs = controller.format_prompt(prompt)

    print(f"\n=== Test Case {idx} ({concept_type}) ===")
    print("Prompt:\n", prompt)

    print("\n--- Output (No Control) ---")
    gen1 = controller.generate(inputs, max_new_tokens=num_new_tokens, do_sample=False)
    print(gen1[len(inputs):])

    print(f"\n--- Output (+ {concept_type} Control) ---")
    gen2 = controller.generate(inputs, layers_to_control=layer_id, control_coef=coeff,
                               max_new_tokens=num_new_tokens, do_sample=False)
    print(gen2[len(inputs):])


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Re-state the following program. Do not include an explanation.

```python
def twoSum(nums, target):
    map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in map:
            return [map[complement], i]
        map[num] = i
    return []
```<|eot_id|>
===== No Control =====


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

```python
def twoSum(nums, target):
    num_map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_map:
            return [num_map[complement], i]
        num_map[num] = i
    return []
```<|eot_id|>

===== + python Control =====
<|start_header_id|>assistant<|end_header_id|>

Here's the revised code:

```python
def twoSum(nums, target):
    map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in map:
            return [map[complement], i]
        map[num] = i
    return []
```

I've made a few adjustments to the code to make it more Pythonic and idiomatic. Here's the revised code:

```python
def twoSum(nums, target):
    map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in map:
            return [map[complement], i]
        map[num] = i
    return []
``

